In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

class CloudRemovalDataset(Dataset):
    """
    Dataset para Cloud Removal (Eliminación de Nubes)
    - Input: Imagen con nubes
    - Output: Imagen limpia (sin nubes)
    """
    def __init__(self, cloudy_dir, clear_dir, size=(256, 256)):
        """
        Args:
            cloudy_dir: Directorio con imágenes nubladas
            clear_dir: Directorio con imágenes limpias (ground truth)
            size: Tamaño de redimensionamiento
        """
        self.cloudy_dir = cloudy_dir
        self.clear_dir = clear_dir
        self.size = size

        # Obtener lista de archivos
        self.cloudy_files = sorted([f for f in os.listdir(cloudy_dir) if f.endswith(('.tif', '.jpg', '.png'))])
        self.clear_files = sorted([f for f in os.listdir(clear_dir) if f.endswith(('.tif', '.jpg', '.png'))])

        # Transformaciones para ambas imágenes (misma normalización)
        self.transform = transforms.Compose([
            transforms.Resize(self.size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.cloudy_files)

    def __getitem__(self, idx):
        # Rutas de imagen nublada y limpia
        cloudy_path = os.path.join(self.cloudy_dir, self.cloudy_files[idx])
        clear_path = os.path.join(self.clear_dir, self.clear_files[idx])

        # Cargar imágenes
        cloudy_image = Image.open(cloudy_path).convert('RGB')
        clear_image = Image.open(clear_path).convert('RGB')

        # Aplicar transformaciones
        cloudy_tensor = self.transform(cloudy_image)
        clear_tensor = self.transform(clear_image)

        return cloudy_tensor, clear_tensor

# ============================================
# DIAGNÓSTICO Y CONFIGURACIÓN
# ============================================
print("=" * 70)
print("CLOUD REMOVAL - DIAGNÓSTICO DEL DATASET")
print("=" * 70)

# Crear dataset
train_dataset = CloudRemovalDataset(
    cloudy_dir="../data/masked",      # Imágenes con nubes
    clear_dir="../data/temporal",         # Imágenes limpias
    size=(256, 256)
)

# Verificar directorios
print(f"\nDirectorio de imágenes nubladas: ../data/temporal")
print(f"  Número de imágenes: {len(train_dataset.cloudy_files)}")
if len(train_dataset.cloudy_files) > 0:
    print(f"  Primeras 5: {train_dataset.cloudy_files[:5]}")

print(f"\nDirectorio de imágenes limpias: ../data/masked")
print(f"  Número de imágenes: {len(train_dataset.clear_files)}")
if len(train_dataset.clear_files) > 0:
    print(f"  Primeras 5: {train_dataset.clear_files[:5]}")

# Verificar coincidencia
if len(train_dataset.cloudy_files) == len(train_dataset.clear_files):
    print(f"\n✓ COINCIDENCIA: Mismo número de archivos ({len(train_dataset)})")
else:
    print(f"\n✗ ERROR: Diferente número de archivos!")
    print(f"  Nubladas: {len(train_dataset.cloudy_files)}")
    print(f"  Limpias: {len(train_dataset.clear_files)}")

# Verificar que los nombres coincidan
print("\nVerificando nombres de archivos:")
all_match = True
for cloudy_file, clear_file in zip(train_dataset.cloudy_files[:5], train_dataset.clear_files[:5]):
    match = "✓" if cloudy_file == clear_file else "✗"
    print(f"  {match} {cloudy_file} <-> {clear_file}")
    if cloudy_file != clear_file:
        all_match = False

if all_match and len(train_dataset.cloudy_files) > 0:
    print("\n✓ Los nombres de archivo coinciden perfectamente")

print("\n" + "=" * 70)

# ============================================
# DATALOADER
# ============================================
batch_size = 2
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

print(f"Dataset de entrenamiento: {len(train_dataset)} pares de imágenes")
print(f"Tamaño de batch: {batch_size}")
print(f"Batches por época: {len(train_dataset) // batch_size}")
print()

# Probar un batch
for cloudy_batch, clear_batch in train_loader:
    print("=" * 70)
    print("PRUEBA DEL PRIMER BATCH")
    print("=" * 70)
    print(f"Imágenes nubladas: {cloudy_batch.shape}")
    print(f"  Rango de valores: [{cloudy_batch.min():.3f}, {cloudy_batch.max():.3f}]")
    print(f"\nImágenes limpias: {clear_batch.shape}")
    print(f"  Rango de valores: [{clear_batch.min():.3f}, {clear_batch.max():.3f}]")
    print("=" * 70)
    break

CLOUD REMOVAL - DIAGNÓSTICO DEL DATASET

Directorio de imágenes nubladas: ../data/temporal
  Número de imágenes: 680
  Primeras 5: ['118032_101.tif', '118032_102.tif', '118032_103.tif', '118032_104.tif', '118032_105.tif']

Directorio de imágenes limpias: ../data/masked
  Número de imágenes: 680
  Primeras 5: ['118032_101.tif', '118032_102.tif', '118032_103.tif', '118032_104.tif', '118032_105.tif']

✓ COINCIDENCIA: Mismo número de archivos (680)

Verificando nombres de archivos:
  ✓ 118032_101.tif <-> 118032_101.tif
  ✓ 118032_102.tif <-> 118032_102.tif
  ✓ 118032_103.tif <-> 118032_103.tif
  ✓ 118032_104.tif <-> 118032_104.tif
  ✓ 118032_105.tif <-> 118032_105.tif

✓ Los nombres de archivo coinciden perfectamente

Dataset de entrenamiento: 680 pares de imágenes
Tamaño de batch: 2
Batches por época: 340

PRUEBA DEL PRIMER BATCH
Imágenes nubladas: torch.Size([2, 3, 256, 256])
  Rango de valores: [-2.118, 2.640]

Imágenes limpias: torch.Size([2, 3, 256, 256])
  Rango de valores: [-2.101, 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_msssim as ms

# ============================
# BLOQUES BÁSICOS
# ============================

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.norm = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        x = F.relu(self.norm(self.conv1(x)))
        x = self.norm(self.conv2(x))
        return F.relu(x + residual)


class AttentionBlock(nn.Module):
    def __init__(self, g_ch, x_ch, inter_ch):
        super().__init__()
        self.Wg = nn.Conv2d(g_ch, inter_ch, 1)
        self.Wx = nn.Conv2d(x_ch, inter_ch, 1)
        self.psi = nn.Conv2d(inter_ch, 1, 1)

    def forward(self, g, x):
        psi = torch.relu(self.Wg(g) + self.Wx(x))
        psi = torch.sigmoid(self.psi(psi))
        return x * psi


# ============================
# ENCODER / DECODER
# ============================

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            ResidualBlock(out_ch)
        )
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.conv(x)
        p = self.pool(x)
        return x, p


class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2)
        self.att = AttentionBlock(out_ch, out_ch, out_ch//2)
        self.conv = nn.Sequential(
            nn.Conv2d(out_ch*2, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            ResidualBlock(out_ch)
        )

    def forward(self, x, skip):
        x = self.up(x)
        skip = self.att(x, skip)
        x = torch.cat([x, skip], dim=1)
        return self.conv(x)


# ============================
# MODELO FINAL
# ============================

class CloudRemovalNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.d1 = Down(3, 32)
        self.d2 = Down(32, 64)
        self.d3 = Down(64, 128)
        self.d4 = Down(128, 256)

        self.bridge = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(512)
        )


        self.u1 = Up(512, 256)
        self.u2 = Up(256, 128)
        self.u3 = Up(128, 64)
        self.u4 = Up(64, 32)

        self.out = nn.Conv2d(32, 3, 1)

    def forward(self, x):
        s1, p1 = self.d1(x)
        s2, p2 = self.d2(p1)
        s3, p3 = self.d3(p2)
        s4, p4 = self.d4(p3)

        b = self.bridge(p4)

        x = self.u1(b, s4)
        x = self.u2(x, s3)
        x = self.u3(x, s2)
        x = self.u4(x, s1)

        return torch.tanh(self.out(x))  # salida normalizada

class CloudLoss(nn.Module):
    def __init__(self, alpha=0.8):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.alpha = alpha

    def forward(self, pred, target):
        l1_loss = self.l1(pred, target)
        ssim_loss = 1 - ms.ssim(pred, target, data_range=2.0, size_average=True)
        return self.alpha * l1_loss + (1 - self.alpha) * ssim_loss


### correr en la nube

In [5]:
import torch.optim as optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CloudRemovalNet().to(device)
criterion = CloudLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=50,
    eta_min=1e-6
)

scaler = torch.cuda.amp.GradScaler()

epochs = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

    for cloudy, clear in loop:
        cloudy = cloudy.to(device)
        clear = clear.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            output = model(cloudy)
            loss = criterion(output, clear)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    scheduler.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")

    # Guardar checkpoints
    if (epoch+1) % 10 == 0:
        torch.save(model.state_dict(), f"cloud_model_epoch_{epoch+1}.pth")


C:\Users\Amalia\AppData\Local\Temp\ipykernel_16304\91141678.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\Amalia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\cuda\amp\grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(
Epoch [1/100]:   0%|          | 0/170 [00:00<?, ?it/s]C:\Users\Amalia\AppData\Local\Temp\ipykernel_16304\91141678.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
C:\Users\Amalia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\cuda\amp\autocast_mode.py:54: UserWarning: CUDA is not avai

KeyboardInterrupt: 

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import torch
import os

torch.set_num_threads(4)  
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"


device = "cpu"   

model = CloudRemovalNet().to(device)
criterion = CloudLoss()

optimizer = optim.Adam(
    model.parameters(),
    lr=2e-4   # un poco mayor para CPU
)

# gradient accumulation para simular batch grande
accum_steps = 4   

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

    for i, (cloudy, clear) in enumerate(loop):
        cloudy = cloudy.to(device)
        clear = clear.to(device)

        output = model(cloudy)
        loss = criterion(output, clear)
        loss = loss / accum_steps

        loss.backward()

        if (i + 1) % accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accum_steps
        loop.set_postfix(loss=loss.item() * accum_steps)

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")

    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f"cloud_cpu_epoch_{epoch+1}.pth")


Epoch [1/60]: 100%|██████████| 340/340 [18:32<00:00,  3.27s/it, loss=0.899]


Epoch 1 | Loss: 0.7775


Epoch [2/60]: 100%|██████████| 340/340 [18:13<00:00,  3.22s/it, loss=0.547]


Epoch 2 | Loss: 0.7299


Epoch [3/60]: 100%|██████████| 340/340 [18:39<00:00,  3.29s/it, loss=0.719]


Epoch 3 | Loss: 0.7207


Epoch [4/60]:  35%|███▍      | 118/340 [07:17<11:00,  2.98s/it, loss=0.525]

In [ ]:
def denorm(x):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1).to(x.device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1).to(x.device)
    return x * std + mean


model.eval()
with torch.no_grad():
    cloudy, clear = next(iter(train_loader))
    cloudy = cloudy.to(device)

    pred = model(cloudy)

    cloudy = denorm(cloudy)
    pred = denorm(pred)
    clear = denorm(clear.to(device))

    fig, axs = plt.subplots(3,3, figsize=(9,9))
    for i in range(3):
        axs[i,0].imshow(cloudy[i].permute(1,2,0).cpu().clamp(0,1))
        axs[i,1].imshow(pred[i].permute(1,2,0).cpu().clamp(0,1))
        axs[i,2].imshow(clear[i].permute(1,2,0).cpu().clamp(0,1))

        axs[i,0].set_title("Cloudy")
        axs[i,1].set_title("Predicted")
        axs[i,2].set_title("Clear")

    plt.show()
